In [115]:
import cv2
from ultralytics import YOLO
import mediapipe as mp
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [112]:
image_frame_path = 'squat.jpg'
frame = cv2.imread(image_frame_path)

In [114]:
model = YOLO('yolov5s.pt')  # YOLOv5 small model
results = model('squat.jpg')
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0].tolist()  # Ensure values are unpacked correctly
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
        cropped_img = frame[int(y1):int(y2), int(x1):int(x2)]
        cropped_img_path = 'cropped_image1.jpg'
        cv2.imwrite(cropped_img_path, cropped_img)


#cv2.imwrite('bounding_box_image.jpg', frame)
cv2.imwrite('cropped_image1.jpg', cropped_img)

PRO TIP  Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.


image 1/1 C:\Users\Hp\Desktop\Semester7\Deep learning\A2_Q1\squat.jpg: 416x640 1 person, 96.8ms
Speed: 0.5ms preprocess, 96.8ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 640)


True

In [96]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)

# Process the image to extract landmarks
img = cv2.imread('cropped_image.jpg')
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
results = pose.process(img_rgb)

# Initialize a dictionary to store the landmarks
landmark_data = {}

# If landmarks are detected, store them in the dictionary
if results.pose_landmarks:
    landmarks = results.pose_landmarks.landmark
    
    for idx, landmark in enumerate(landmarks):
        name = mp_pose.PoseLandmark(idx).name.lower()  # Get landmark name (e.g., 'nose')
        # Store x, y, z coordinates in the dictionary with appropriate naming
        landmark_data[f'x_{name}'] = landmark.x
        landmark_data[f'y_{name}'] = landmark.y
        landmark_data[f'z_{name}'] = landmark.z

# Convert the dictionary into a DataFrame (with one row)
df = pd.DataFrame([landmark_data])

df.head()  # Display the extracted landmarks


C:\Users\Hp\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


,x_nose,y_nose,z_nose,x_left_eye_inner,y_left_eye_inner,z_left_eye_inner,x_left_eye,y_left_eye,z_left_eye,x_left_eye_outer,...,z_left_heel,x_right_heel,y_right_heel,z_right_heel,x_left_foot_index,y_left_foot_index,z_left_foot_index,x_right_foot_index,y_right_foot_index,z_right_foot_index
0,0.72771,0.11225,-0.549454,0.705804,0.088476,-0.497974,0.70392,0.087877,-0.498356,0.701882,...,0.211859,0.243534,0.93608,-1.136549,0.777628,0.85736,0.068203,0.507565,0.960548,-1.341813


In [97]:
landmarks_df = pd.read_csv('landmarks.csv')       # Contains 3D body landmark coordinates

In [98]:
scaler = StandardScaler()
landmark_columns = [col for col in landmarks_df.columns if 'x' in col or 'y' in col or 'z' in col]
landmarks_scaled = scaler.fit_transform(landmarks_df[landmark_columns])

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels_df['pose'])

labels_encoded_df = pd.DataFrame(labels_encoded, columns=['encoded_pose'])
landmarks_scaled_df = pd.DataFrame(landmarks_scaled, columns=landmark_columns)

In [99]:
landmarks_scaled_df.head()

,x_nose,y_nose,z_nose,x_left_eye_inner,y_left_eye_inner,z_left_eye_inner,x_left_eye,y_left_eye,z_left_eye,x_left_eye_outer,...,z_left_heel,x_right_heel,y_right_heel,z_right_heel,x_left_foot_index,y_left_foot_index,z_left_foot_index,x_right_foot_index,y_right_foot_index,z_right_foot_index
0,-0.289783,-0.661234,-0.258839,-0.247921,-0.752400,-0.253219,-0.204987,-0.772513,-0.253090,-0.164902,...,0.431417,0.154649,-0.012087,0.322033,0.117226,0.189793,0.483646,0.193902,0.184172,0.343463
1,-0.220475,-0.834092,-1.794383,-0.179172,-0.888243,-1.703873,-0.160427,-0.896286,-1.703878,-0.142853,...,-0.613358,0.137696,0.675948,-0.533809,0.053341,0.811645,-0.901463,0.017538,0.824816,-0.844322
2,-0.162086,-0.794096,-1.950812,-0.123542,-0.844862,-1.848109,-0.101850,-0.851344,-1.848070,-0.082342,...,-0.698363,0.107235,0.554816,-0.662936,-0.014769,0.726835,-1.022749,0.039247,0.722980,-0.998265
3,-0.219927,-0.750420,-1.907641,-0.184936,-0.803566,-1.808319,-0.166758,-0.812016,-1.808373,-0.153472,...,-0.506300,0.130772,0.850474,-0.409875,-0.016306,0.901924,-0.817274,0.035139,0.905790,-0.720888
4,-0.074141,-0.613636,-0.187536,-0.059495,-0.612407,-0.132464,-0.050922,-0.612968,-0.132698,-0.041771,...,0.288225,0.094769,0.610570,0.253087,0.009414,0.551468,0.166520,0.112885,0.571181,0.130163


In [100]:
labels_encoded_df

,encoded_pose
0,0
1,0
2,0
3,0
4,0
...,...
1367,9
1368,9
1369,9
1370,9


In [101]:
combined_df = pd.concat([landmarks_scaled_df], axis=1)

In [102]:
combined_df

,x_nose,y_nose,z_nose,x_left_eye_inner,y_left_eye_inner,z_left_eye_inner,x_left_eye,y_left_eye,z_left_eye,x_left_eye_outer,...,z_left_heel,x_right_heel,y_right_heel,z_right_heel,x_left_foot_index,y_left_foot_index,z_left_foot_index,x_right_foot_index,y_right_foot_index,z_right_foot_index
0,-0.289783,-0.661234,-0.258839,-0.247921,-0.752400,-0.253219,-0.204987,-0.772513,-0.253090,-0.164902,...,0.431417,0.154649,-0.012087,0.322033,0.117226,0.189793,0.483646,0.193902,0.184172,0.343463
1,-0.220475,-0.834092,-1.794383,-0.179172,-0.888243,-1.703873,-0.160427,-0.896286,-1.703878,-0.142853,...,-0.613358,0.137696,0.675948,-0.533809,0.053341,0.811645,-0.901463,0.017538,0.824816,-0.844322
2,-0.162086,-0.794096,-1.950812,-0.123542,-0.844862,-1.848109,-0.101850,-0.851344,-1.848070,-0.082342,...,-0.698363,0.107235,0.554816,-0.662936,-0.014769,0.726835,-1.022749,0.039247,0.722980,-0.998265
3,-0.219927,-0.750420,-1.907641,-0.184936,-0.803566,-1.808319,-0.166758,-0.812016,-1.808373,-0.153472,...,-0.506300,0.130772,0.850474,-0.409875,-0.016306,0.901924,-0.817274,0.035139,0.905790,-0.720888
4,-0.074141,-0.613636,-0.187536,-0.059495,-0.612407,-0.132464,-0.050922,-0.612968,-0.132698,-0.041771,...,0.288225,0.094769,0.610570,0.253087,0.009414,0.551468,0.166520,0.112885,0.571181,0.130163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367,-0.158838,-0.508424,1.186482,-0.161233,-0.498884,1.073936,-0.183135,-0.499448,1.073970,-0.205998,...,-0.705417,0.542900,0.953885,-0.402320,-0.394849,0.847053,-0.518072,0.450328,0.775831,-0.248364
1368,-0.176753,-0.524131,1.019330,-0.180981,-0.516053,0.904663,-0.202893,-0.516836,0.904650,-0.224988,...,-0.628998,0.507750,0.919187,-0.177390,-0.409397,0.733641,-0.434892,0.416522,0.602620,-0.009007
1369,-0.262519,-0.450150,1.338475,-0.247022,-0.465271,1.230194,-0.261439,-0.467006,1.230229,-0.276785,...,-0.720129,0.466703,1.153060,-0.552663,-0.517321,0.960668,-0.538207,0.357259,0.939585,-0.418293
1370,-0.440999,-0.424966,1.447354,-0.462385,-0.420909,1.332767,-0.481807,-0.416772,1.332772,-0.500257,...,-0.532151,0.330014,1.117194,-0.500241,-0.508605,0.832025,-0.486112,0.300858,0.811696,-0.474312


In [103]:
df

,x_nose,y_nose,z_nose,x_left_eye_inner,y_left_eye_inner,z_left_eye_inner,x_left_eye,y_left_eye,z_left_eye,x_left_eye_outer,...,z_left_heel,x_right_heel,y_right_heel,z_right_heel,x_left_foot_index,y_left_foot_index,z_left_foot_index,x_right_foot_index,y_right_foot_index,z_right_foot_index
0,0.72771,0.11225,-0.549454,0.705804,0.088476,-0.497974,0.70392,0.087877,-0.498356,0.701882,...,0.211859,0.243534,0.93608,-1.136549,0.777628,0.85736,0.068203,0.507565,0.960548,-1.341813


In [104]:
common_features = list(set(combined_df.columns).intersection(set(df.columns)))  # Convert set to list
combined_df = combined_df[common_features]
df = df[common_features]

print(f"Common features retained: {common_features}")
print(f"Updated combined_df shape: {combined_df.shape}")
print(f"Updated df shape: {df.shape}")


Common features retained: ['y_left_wrist', 'x_left_elbow', 'x_right_ear', 'z_right_hip', 'x_right_ankle', 'y_right_wrist', 'z_right_elbow', 'x_right_eye_outer', 'y_left_eye', 'z_right_foot_index', 'x_mouth_right', 'x_right_heel', 'x_right_elbow', 'z_left_eye_inner', 'z_left_eye_outer', 'z_left_foot_index', 'y_left_foot_index', 'y_right_knee', 'x_left_foot_index', 'x_left_knee', 'y_right_hip', 'x_left_eye', 'z_left_ear', 'x_left_wrist', 'y_right_eye_inner', 'z_right_eye_inner', 'x_left_shoulder', 'y_right_elbow', 'y_left_eye_inner', 'z_left_heel', 'z_nose', 'y_mouth_left', 'y_left_hip', 'z_left_ankle', 'y_left_knee', 'z_left_eye', 'y_nose', 'x_right_eye_inner', 'x_left_ear', 'y_left_heel', 'z_mouth_right', 'x_nose', 'x_right_hip', 'y_left_ankle', 'x_left_ankle', 'y_mouth_right', 'z_right_eye_outer', 'y_right_foot_index', 'x_left_eye_inner', 'x_left_heel', 'x_right_wrist', 'z_right_heel', 'y_right_heel', 'z_left_elbow', 'z_left_shoulder', 'z_mouth_left', 'x_left_eye_outer', 'z_right_knee

In [105]:
combined_df = pd.concat([combined_df,labels_encoded_df ], axis=1)
combined_df

,y_left_wrist,x_left_elbow,x_right_ear,z_right_hip,x_right_ankle,y_right_wrist,z_right_elbow,x_right_eye_outer,y_left_eye,z_right_foot_index,...,y_left_eye_outer,y_right_ankle,y_left_shoulder,z_right_shoulder,y_right_shoulder,z_left_wrist,y_left_elbow,z_left_hip,z_right_ankle,encoded_pose
0,0.420393,0.147428,-0.355083,-0.070659,0.149955,0.544610,0.068068,-0.356721,-0.772513,0.343463,...,-0.791055,0.048124,-0.595243,-0.155888,-0.467215,0.200301,0.157672,0.070659,0.309747,0
1,0.971410,0.356070,-0.291814,0.086312,0.080713,0.976972,-1.138219,-0.281918,-0.896286,-0.844322,...,-0.902032,0.687470,-0.504418,-1.151831,-0.555443,-1.492563,0.536058,-0.086312,-0.624008,0
2,1.041677,0.499777,-0.259100,0.075210,0.051934,0.996718,-1.278585,-0.228621,-0.851344,-0.998265,...,-0.856940,0.558912,-0.522059,-1.231465,-0.538226,-1.561328,0.676985,-0.075210,-0.763780,0
3,0.661501,0.490251,-0.261978,0.170807,0.091523,0.783665,-1.060530,-0.258796,-0.812016,-0.720888,...,-0.818520,0.900671,-0.580332,-1.163498,-0.479111,-1.370882,0.170868,-0.170807,-0.452923,0
4,0.230725,0.219092,-0.106652,0.025621,0.099171,0.240307,0.134034,-0.101869,-0.612968,0.130163,...,-0.613095,0.649470,-0.558891,0.203120,-0.504554,0.074217,-0.038798,-0.025621,0.255671,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367,-0.440902,-1.171091,0.032064,0.882260,0.568342,-0.455427,1.484925,-0.033649,-0.499448,-0.248364,...,-0.499402,0.986036,-0.536209,0.985901,-0.525171,1.035464,-0.283049,-0.882260,-0.369891,9
1368,-0.432988,-1.184758,0.024424,0.852523,0.530577,-0.386471,1.274471,-0.047803,-0.516836,-0.009007,...,-0.516781,0.934390,-0.516844,0.812333,-0.544459,0.871402,-0.283755,-0.852523,-0.152310,9
1369,-0.443579,-1.202220,-0.063324,0.815750,0.477090,-0.356472,1.710404,-0.145371,-0.467006,-0.418293,...,-0.467933,1.185332,-0.566678,1.182502,-0.493642,1.298693,-0.298709,-0.815750,-0.517566,9
1370,-0.308571,-0.780570,-0.279455,0.181763,0.358436,-0.413218,0.526383,-0.320667,-0.416772,-0.474312,...,-0.411570,1.156359,-0.511548,0.864024,-0.545933,-0.150630,-0.346680,-0.181763,-0.476180,9


In [106]:
X = combined_df.drop('encoded_pose', axis=1).values  # Drop the label column
y = combined_df['encoded_pose'].values  # Labels (exercise types)

X = X.reshape(X.shape[0], 3, -1, 1)  # (batch_size, channels, landmarks, 1)

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

train_loader = DataLoader(TensorDataset(X_tensor, y_tensor), batch_size=32, shuffle=True)


In [116]:
class ExerciseCNN(nn.Module):
    def __init__(self, num_classes):
        super(ExerciseCNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 16, kernel_size=(3, 1))  
        self.conv2 = nn.Conv2d(16, 32, kernel_size=(3, 1))  
        self.pool = nn.MaxPool2d(kernel_size=(2, 1))  
        
        self.fc1 = None 
        self.fc2 = nn.Linear(64, num_classes)  

    def _initialize_fc1(self, x):
        num_features = x.view(x.size(0), -1).size(1)  
        self.fc1 = nn.Linear(num_features, 64)  

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  
        x = self.pool(F.relu(self.conv2(x)))  
        
        if self.fc1 is None:
            self._initialize_fc1(x)  
        
        x = x.view(x.size(0), -1)  
        x = F.relu(self.fc1(x)) 
        x = self.fc2(x)  
        return x

num_classes = len(set(y))  
model = ExerciseCNN(num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 200  
for epoch in range(num_epochs):
    model.train()  
    running_loss = 0.0
    
    all_labels = []
    all_preds = []
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()  
        
        outputs = model(inputs)  
        loss = criterion(outputs, labels)  
        
        loss.backward()  
        optimizer.step()  
        
        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)  
        all_labels.extend(labels.cpu().numpy())  
        all_preds.extend(predicted.cpu().numpy()) 
    
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")
    print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

C:\Users\Hp\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Hp\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch [1/200], Loss: 2.2288
Accuracy: 0.1997, Precision: 0.1688, Recall: 0.1997, F1-score: 0.1098
Epoch [2/200], Loss: 2.0802
Accuracy: 0.2690, Precision: 0.1810, Recall: 0.2690, F1-score: 0.1562


C:\Users\Hp\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Hp\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch [3/200], Loss: 1.8680
Accuracy: 0.3710, Precision: 0.4875, Recall: 0.3710, F1-score: 0.2716
Epoch [4/200], Loss: 1.5994
Accuracy: 0.4461, Precision: 0.5346, Recall: 0.4461, F1-score: 0.3694
Epoch [5/200], Loss: 1.3640
Accuracy: 0.5838, Precision: 0.6301, Recall: 0.5838, F1-score: 0.5412
Epoch [6/200], Loss: 1.1974
Accuracy: 0.6545, Precision: 0.6334, Recall: 0.6545, F1-score: 0.6248
Epoch [7/200], Loss: 1.0787
Accuracy: 0.6633, Precision: 0.6567, Recall: 0.6633, F1-score: 0.6339
Epoch [8/200], Loss: 0.9885
Accuracy: 0.6990, Precision: 0.6884, Recall: 0.6990, F1-score: 0.6810
Epoch [9/200], Loss: 0.9193
Accuracy: 0.7223, Precision: 0.7153, Recall: 0.7223, F1-score: 0.7118
Epoch [10/200], Loss: 0.8657
Accuracy: 0.7281, Precision: 0.7214, Recall: 0.7281, F1-score: 0.7184
Epoch [11/200], Loss: 0.8228
Accuracy: 0.7434, Precision: 0.7423, Recall: 0.7434, F1-score: 0.7383
Epoch [12/200], Loss: 0.7805
Accuracy: 0.7485, Precision: 0.7437, Recall: 0.7485, F1-score: 0.7420
Epoch [13/200], L

In [117]:
scaler = StandardScaler()
df_columns = [col for col in df.columns if 'x' in col or 'y' in col or 'z' in col]
df_scaled = scaler.fit_transform(df[df_columns])  
X_test = df_scaled.reshape(df_scaled.shape[0], 3, -1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
test_loader = DataLoader(TensorDataset(X_test_tensor), batch_size=32)

In [118]:
model.eval()  
y_pred = []

with torch.no_grad():
    for inputs in test_loader:
        outputs = model(inputs[0])  
        _, predicted = torch.max(outputs, 1)  
        y_pred.extend(predicted.cpu().numpy())  

predicted_labels = label_encoder.inverse_transform(y_pred)  
for idx, label in enumerate(predicted_labels):
    print(f"Sample {idx}: Predicted Exercise - {label}")

Sample 0: Predicted Exercise - squats_up
